## Extraction of data from [wiki article](https://en.wikipedia.org/wiki/Timeline_of_the_21st_century)

Need to download the xml file on this [link](https://en.wikipedia.org/wiki/Special:Export/Timeline_of_the_21st_century)

This notebook extract the events written on the wiki article: [*Timeline of the 21st century*](https://en.wikipedia.org/wiki/Timeline_of_the_21st_century) from its corresponding xml file (*datasets/wiki_events.xml*). It selects all events registered between 2005 and 2019 and save it as csv file in: *datasets/wiki_events.csv*. The following packages are needed:
- ``` pip install pandas ```
- ``` pip install beautifulsoup4 ```



In [2]:
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
path = '../datasets/wiki_events.xml'
with open(path, 'r') as f:
    data = f.read()

Bs_data = BeautifulSoup(data, "xml")

In [4]:
plain_txt = Bs_data.text
idx = plain_txt.find('===[[2005]]===')
text = plain_txt[idx:]

In [5]:
text = text.replace('\n\n', '\n')
text = text.replace('[', '')
text = text.replace(']', '')

In [6]:
split_txt = text.split('\n')

In [7]:
copy_txt = split_txt.copy()
new_list=[]
for i,txt in enumerate(copy_txt):
    if 'see' in txt:
        continue
    if 'See' in txt:
        break
    if  '2020s' in txt:
        continue
    if  '2010s' in txt:
        continue
    if '--' in txt:
        continue
    if '===' not in txt:
        new_list.append(txt[2:])
    else:
        new_list.append(txt)

In [8]:
list_months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

In [9]:
list_tuples = []
for i,element in enumerate(new_list):
    month = ''
    if '===' in element:
        year = element[3:7]
        continue
    if any(month in element for month in list_months):
        idx = element.find(':')
        month = element[:idx]
        element=element[idx+1:]
    list_tuples.append((year,month, element))

In [10]:
events_df = pd.DataFrame(list_tuples, columns =['Year','Day', 'Event'])

In [11]:
events_df.to_csv('../datasets/wiki_events.csv')